In [1]:
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/imaginary-formula-1/formula1.csv


In [2]:
data = pd.read_csv('/kaggle/input/imaginary-formula-1/formula1.csv')
data

,Unnamed: 0,name,country,team,races,years_in_f1,championships,gp_wins,podiums,poles,dnfs,fav_circuit,avg_yearly_salary
0,0,Frank Daniels,United States,Mercedes,67,4,5,56,29,62,16,Albert Park,16
1,1,Pierangelo Franceschi,Italy,Aston Martin,218,10,3,75,43,55,9,Circuit Zandvoort,17
2,2,Frederick Gamble,United States,Ferrari,167,8,3,56,52,72,11,Imola,14
3,3,Tristan Rousseau-Grondin,France,AlphaTauri,251,12,4,68,37,70,16,Circuit of The Americas,14
4,4,Horacio Alemany,Spain,Aston Martin,185,9,1,73,37,66,11,Jeddah Corniche Circuit,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,745,Ottone Cuda,Italy,AlphaTauri,193,9,1,84,24,63,15,Red Bull Ring,10
746,746,Leonard Murphy-Wells,Great Britain,Ferrari,133,7,5,70,56,76,12,Suzuka Circuit,16
747,747,Veerle Vos,Netherlands,Ferrari,206,10,2,60,29,58,9,Hungaroring,11
748,748,Pamela White,Great Britain,Haas,166,8,5,52,38,69,13,Circuit Zandvoort,11


In [3]:
data = data.drop('Unnamed: 0',axis=1)

In [4]:
data

,name,country,team,races,years_in_f1,championships,gp_wins,podiums,poles,dnfs,fav_circuit,avg_yearly_salary
0,Frank Daniels,United States,Mercedes,67,4,5,56,29,62,16,Albert Park,16
1,Pierangelo Franceschi,Italy,Aston Martin,218,10,3,75,43,55,9,Circuit Zandvoort,17
2,Frederick Gamble,United States,Ferrari,167,8,3,56,52,72,11,Imola,14
3,Tristan Rousseau-Grondin,France,AlphaTauri,251,12,4,68,37,70,16,Circuit of The Americas,14
4,Horacio Alemany,Spain,Aston Martin,185,9,1,73,37,66,11,Jeddah Corniche Circuit,14
...,...,...,...,...,...,...,...,...,...,...,...,...
745,Ottone Cuda,Italy,AlphaTauri,193,9,1,84,24,63,15,Red Bull Ring,10
746,Leonard Murphy-Wells,Great Britain,Ferrari,133,7,5,70,56,76,12,Suzuka Circuit,16
747,Veerle Vos,Netherlands,Ferrari,206,10,2,60,29,58,9,Hungaroring,11
748,Pamela White,Great Britain,Haas,166,8,5,52,38,69,13,Circuit Zandvoort,11


In [5]:
data['country'].value_counts()

Great Britain    259
Italy            114
Finland           82
Spain             79
France            71
Australia         41
Netherlands       40
United States     34
Germany           30
Name: country, dtype: int64

In [6]:
data['team'].value_counts()

Ferrari         194
McLaren         151
Alfa Romeo      147
Aston Martin     91
AlphaTauri       90
Alpine           34
Williams         21
Haas             13
Mercedes          8
Red Bull          1
Name: team, dtype: int64

In [7]:
data.corr()['avg_yearly_salary'].sort_values(ascending=False)


avg_yearly_salary    1.000000
championships        0.546401
gp_wins              0.390814
podiums              0.304301
poles                0.243704
races                0.165393
years_in_f1          0.158370
dnfs                 0.073816
Name: avg_yearly_salary, dtype: float64

In [8]:
features = ['championships','gp_wins','podiums','poles','races','years_in_f1']

In [9]:
categoricals = ['team']
data_ohe = pd.get_dummies(data, columns = categoricals)
data_ohe

,name,country,races,years_in_f1,championships,gp_wins,podiums,poles,dnfs,fav_circuit,...,team_Alfa Romeo,team_AlphaTauri,team_Alpine,team_Aston Martin,team_Ferrari,team_Haas,team_McLaren,team_Mercedes,team_Red Bull,team_Williams
0,Frank Daniels,United States,67,4,5,56,29,62,16,Albert Park,...,0,0,0,0,0,0,0,1,0,0
1,Pierangelo Franceschi,Italy,218,10,3,75,43,55,9,Circuit Zandvoort,...,0,0,0,1,0,0,0,0,0,0
2,Frederick Gamble,United States,167,8,3,56,52,72,11,Imola,...,0,0,0,0,1,0,0,0,0,0
3,Tristan Rousseau-Grondin,France,251,12,4,68,37,70,16,Circuit of The Americas,...,0,1,0,0,0,0,0,0,0,0
4,Horacio Alemany,Spain,185,9,1,73,37,66,11,Jeddah Corniche Circuit,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,Ottone Cuda,Italy,193,9,1,84,24,63,15,Red Bull Ring,...,0,1,0,0,0,0,0,0,0,0
746,Leonard Murphy-Wells,Great Britain,133,7,5,70,56,76,12,Suzuka Circuit,...,0,0,0,0,1,0,0,0,0,0
747,Veerle Vos,Netherlands,206,10,2,60,29,58,9,Hungaroring,...,0,0,0,0,1,0,0,0,0,0
748,Pamela White,Great Britain,166,8,5,52,38,69,13,Circuit Zandvoort,...,0,0,0,0,0,1,0,0,0,0


In [10]:
data_ohe = data_ohe.drop(['name','country','fav_circuit'],axis=1)
scaler = MinMaxScaler()
model=scaler.fit(data_ohe)
scaled=model.transform(data_ohe)

data_scaled = pd.DataFrame(scaled, index=data_ohe.index, columns=data_ohe.columns)
data_scaled.head()

,races,years_in_f1,championships,gp_wins,podiums,poles,dnfs,avg_yearly_salary,team_Alfa Romeo,team_AlphaTauri,team_Alpine,team_Aston Martin,team_Ferrari,team_Haas,team_McLaren,team_Mercedes,team_Red Bull,team_Williams
0,0.167857,0.230769,0.625,0.371429,0.322222,0.457143,0.733333,0.733333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.707143,0.692308,0.375,0.642857,0.477778,0.357143,0.266667,0.800000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.525000,0.538462,0.375,0.371429,0.577778,0.600000,0.400000,0.600000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.825000,0.846154,0.500,0.542857,0.411111,0.571429,0.733333,0.600000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.589286,0.615385,0.125,0.614286,0.411111,0.514286,0.400000,0.600000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X = data_scaled.drop(['avg_yearly_salary'],axis=1)
y = data_scaled['avg_yearly_salary']


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((525, 17), (225, 17), (525,), (225,))

In [12]:
regressor = LinearRegression()
regressor.fit(X_train,y_train)

predictions = regressor.predict(X_test)

In [13]:
MSE = np.square(np.subtract(y_test,predictions)).mean() 

RMSE = math.sqrt(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

Root Mean Square Error:

0.04550307688990221
